In [3]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

# --- 1. Load Data (Assumes PIDD dataset is available as 'diabetes.csv') ---
# Replace 'diabetes.csv' with the actual path/name of your file
try:
    df = pd.read_csv('diabetes.csv')
except FileNotFoundError:
    print("Please make sure 'diabetes.csv' is in the current directory or provide the correct path.")
    # Creating dummy data for demonstration if file not found
    data = {
        'Pregnancies': [6, 1, 8, 1, 0, 5],
        'Glucose': [148, 85, 183, 89, 137, 116],
        'BloodPressure': [72, 66, 64, 66, 40, 74],
        'SkinThickness': [35, 29, 0, 23, 35, 0],
        'Insulin': [0, 0, 0, 94, 168, 0],
        'BMI': [33.6, 26.6, 23.3, 28.1, 43.1, 25.6],
        'DiabetesPedigreeFunction': [0.627, 0.351, 0.672, 0.167, 2.288, 0.201],
        'Age': [50, 31, 32, 21, 33, 30],
        'Outcome': [1, 0, 1, 0, 1, 0]
    }
    df = pd.DataFrame(data)

# --- 2. Data Cleaning / Preprocessing ---

# Identify columns where 0 is an invalid/missing value
zero_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Replace 0s with the median of the respective column (a robust imputation technique)
for col in zero_cols:
    df[col] = df[col].replace(0, df[col].median())

# Separate features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Standardize the features (important for most ML algorithms)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- 3. Model Training: Gradient Boosting Classifier ---

# Initialize the GBC model with a starting set of parameters
gb_model = GradientBoostingClassifier(
    n_estimators=100,      # Number of boosting stages/trees
    learning_rate=0.1,     # Shrinkage (step size)
    max_depth=3,           # Depth of individual trees
    random_state=42
)

print("\n--- Training Gradient Boosting Classifier ---")
gb_model.fit(X_train_scaled, y_train)


# --- 4. Prediction and Evaluation ---

# Make predictions on the test set
y_pred = gb_model.predict(X_test_scaled)
y_pred_proba = gb_model.predict_proba(X_test_scaled)[:, 1] # Probability for the positive class (1)

# Print evaluation metrics
print("\n--- Model Evaluation (Initial GBC) ---")
print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.4f}")
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

# --- 5. Hyperparameter Tuning (Optional but recommended) ---

# Define the grid of parameters to search
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_leaf': [1, 2]
}

# Use GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=GradientBoostingClassifier(random_state=42),
    param_grid=param_grid,
    scoring='roc_auc',  # Use ROC-AUC as the primary metric for optimization
    cv=5,
    verbose=1,
    n_jobs=-1
)

print("\n--- Starting Grid Search for Hyperparameter Tuning ---")
grid_search.fit(X_train_scaled, y_train)

# Get the best model
best_gb_model = grid_search.best_estimator_
print(f"\nBest Parameters found: {grid_search.best_params_}")

# Evaluate the best model
best_y_pred = best_gb_model.predict(X_test_scaled)
best_y_pred_proba = best_gb_model.predict_proba(X_test_scaled)[:, 1]

print("\n--- Model Evaluation (Tuned GBC) ---")
print(f"Accuracy Score: {accuracy_score(y_test, best_y_pred):.4f}")
print(f"ROC-AUC Score: {roc_auc_score(y_test, best_y_pred_proba):.4f}")
print("\nClassification Report (Tuned):\n", classification_report(y_test, best_y_pred))
print("\nConfusion Matrix (Tuned):\n", confusion_matrix(y_test, best_y_pred))
# [[TN, FP]
#  [FN, TP]





--- Training Gradient Boosting Classifier ---

--- Model Evaluation (Initial GBC) ---
Accuracy Score: 0.7597
ROC-AUC Score: 0.8278

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.85      0.82       100
           1       0.68      0.59      0.63        54

    accuracy                           0.76       154
   macro avg       0.74      0.72      0.73       154
weighted avg       0.75      0.76      0.76       154


Confusion Matrix:
 [[85 15]
 [22 32]]

--- Starting Grid Search for Hyperparameter Tuning ---
Fitting 5 folds for each of 36 candidates, totalling 180 fits

Best Parameters found: {'learning_rate': 0.05, 'max_depth': 3, 'min_samples_leaf': 1, 'n_estimators': 100}

--- Model Evaluation (Tuned GBC) ---
Accuracy Score: 0.7403
ROC-AUC Score: 0.8204

Classification Report (Tuned):
               precision    recall  f1-score   support

           0       0.78      0.84      0.81       100
           1       0.65   

In [4]:
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

GROUP_NUMBER = "05"
try:
    
    model_filename = f'GROUP_{GROUP_NUMBER}.pkl'
    with open(model_filename, 'wb') as file:
        pickle.dump(best_gb_model, file)
    print(f"✅ Model successfully saved using pickle as: {model_filename}")
    scaler_filename = f'GROUP_{GROUP_NUMBER}.pkl'
    with open(scaler_filename, 'wb') as file:
        pickle.dump(scaler, file)
    
    print(f"✅ Scaler successfully saved using pickle as: {scaler_filename}")
except NameError:
    # This block executes if 'best_gb_model' or 'scaler' were not defined 
    # in the current Python environment (common outside of a full notebook run)
    print("⚠️ Error: 'best_gb_model' or 'scaler' objects were not found.")
    print("Please ensure you have run your model training and scaling steps before saving.")
except Exception as e:
    print(f"An unexpected error occurred during saving: {e}")

✅ Model successfully saved using pickle as: GROUP_05.pkl
✅ Scaler successfully saved using pickle as: GROUP_05.pkl
